Installation préalables

In [16]:
# !pip install plotly
# !pip install nominatim
# !pip install geopy


Importations des librairies

In [17]:
import numpy as np
import pandas as pd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from geopy.geocoders import Nominatim
import csv
import json


Le code

In [18]:
annees =[]

# récupération des données
with open('dataSetPascal.json') as json_data:
    data = json.load(json_data) # type = list


# trouver année min et année max pour calibrer la barre
for dict in data:
  try:
    if(dict["ddn"] != ""):
      annees.append(dict["ddn"])
    if(dict["ddm"] != ""):
      annees.append(dict["ddm"])
  except:
    pass

print("année début : " + min(annees), "\nannée fin : " + max(annees))


année début : 1746 
année fin : 1920


In [19]:
# dictionnaires
dict_datetime = {}
dict_info = {}
dict_lat = {}
dict_lon = {}
dict_etat = {}

geocoder = Nominatim(user_agent="http")

# on set les clés des dict (ce sont les années), on ne s'intéresse qu'à l'intervalle [année min, année max]
# set les valeurs des clés des lat et long
c = 0
for dict in data:
  
  for k in range(int(min(annees)),int(max(annees))+1):
    try:
      dict_datetime[c] = str(k)
      dict_etat[c] = "NA"
    except:
      pass
    dict_info[c] = dict["nom"] + " " + dict["prenom"]
    c+=1
      

  try:
    ville_naissance = geocoder.geocode(dict["ldn"])  
    dict_lat[int(dict["ddn"])] = str(ville_naissance.latitude)
    dict_lon[int(dict["ddn"])] = str(ville_naissance.longitude)
    dict_etat[int(dict["ddn"])] = "Naissance"
    #print(dict["ldn"] + " : " + str(ville_naissance.latitude) + " " + str(ville_naissance.longitude))
  except:
    #print("non 1")
    pass

  try:
    ville_mort = geocoder.geocode(dict["ldm"])
    dict_info[int(dict["ddn"])] = dict["nom"] + " " + dict["prenom"]
    dict_lat[int(dict["ddm"])] = str(ville_mort.latitude)
    dict_lon[int(dict["ddm"])] = str(ville_mort.longitude)
    dict_etat[int(dict["ddm"])] = "Mort"
    #print(dict["ldm"] + " : " + str(ville_mort.latitude) + " " + str(ville_mort.longitude))
  except:
    #print("non 2")
    pass




'''
print(len(dict_datetime))
print(len(dict_lat_naissances))
print(len(dict_lon_naissances))
print(len(dict_lat_morts))
print(len(dict_lon_morts))

print(dict_datetime)
print(dict_lat_naissances)
print(dict_lon_naissances)
print(dict_lat_morts)
print(dict_lon_morts)
'''

df = pd.DataFrame({'datetime': dict_datetime, 'lat': dict_lat, 'lon': dict_lon, 'etat': dict_etat})

print(df)


# Naissances
fig = px.scatter_geo(df, 
                     lat='lat', 
                     lon='lon', 
                     scope='world',
                     projection="mercator", 
                     animation_frame="datetime",
                     animation_group="etat",
                     hover_name="etat",
                     color="etat",
                     title='Affichage des naissances')

fig.update_layout(autosize=False, width=1000, height=1000)
fig.update(layout_coloraxis_showscale=False)

"""
# Morts
fig2 = px.scatter_geo(df, 
                     lat='lat_morts', 
                     lon='lon_morts', 
                     scope='world',
                     projection="mercator", 
                     animation_frame="datetime",
                     title='Affichage des morts',
                     )


fig2.update_layout(autosize=False, width=1000, height=1000)
fig2.update(layout_coloraxis_showscale=False)

fig.add_trace(fig2.data[-1])
"""
fig.write_html("./Confirmed.html")
fig.show()



     datetime         lat        lon       etat
0        1746         NaN        NaN         NA
1        1747  41.9263991  8.7376029  Naissance
2        1748  41.9263991  8.7376029  Naissance
3        1749  41.9263991  8.7376029  Naissance
4        1750  41.9263991  8.7376029  Naissance
...       ...         ...        ...        ...
3495     1916         NaN        NaN         NA
3496     1917         NaN        NaN         NA
3497     1918         NaN        NaN         NA
3498     1919         NaN        NaN         NA
3499     1920         NaN        NaN         NA

[3500 rows x 4 columns]


In [20]:

# geocoder = Nominatim(user_agent="http")


df = px.data.gapminder()
print(df)



fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     animation_frame="year", projection="natural earth")

fig.update_layout(autosize=False, width=1000, height=1000)
fig.update(layout_coloraxis_showscale=False)

# fig.write_html("./Confirmed.html")
fig.show()


          country continent  year  lifeExp       pop   gdpPercap iso_alpha  \
0     Afghanistan      Asia  1952   28.801   8425333  779.445314       AFG   
1     Afghanistan      Asia  1957   30.332   9240934  820.853030       AFG   
2     Afghanistan      Asia  1962   31.997  10267083  853.100710       AFG   
3     Afghanistan      Asia  1967   34.020  11537966  836.197138       AFG   
4     Afghanistan      Asia  1972   36.088  13079460  739.981106       AFG   
...           ...       ...   ...      ...       ...         ...       ...   
1699     Zimbabwe    Africa  1987   62.351   9216418  706.157306       ZWE   
1700     Zimbabwe    Africa  1992   60.377  10704340  693.420786       ZWE   
1701     Zimbabwe    Africa  1997   46.809  11404948  792.449960       ZWE   
1702     Zimbabwe    Africa  2002   39.989  11926563  672.038623       ZWE   
1703     Zimbabwe    Africa  2007   43.487  12311143  469.709298       ZWE   

      iso_num  
0           4  
1           4  
2           4  

In [21]:
mapbox_access_token = "sk.eyJ1IjoiemllZGdvYmppIiwiYSI6ImNsMmc1bnZleDAwOXYzYm13d3ltd2MzbmIifQ.4Upq_YEfiyn4U8UXkrL7PQ"
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon",     color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()